In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.wkt import loads
import geotable
import os

### Script Objectives:

This script filters our two main camera datasets (Amnesty Internation and Hikvision) via spatial joins on Brooklyn's shapefile. This will return a subset of the original camera datasets that we are interested in for our surviellance metric to be created for the neighborhoods across Brooklyn.

#### Input Files:

 **Cameras**

 * Amnesty International Camera Dataset: `os.path.dirname(os.path.dirname(path))+'/CapstoneData/amnesty_camera_counts_per_intersections.csv'`

 * Hikvision Camera Dataset: `../Data/NYC_Surveillance_Map_Hikvision.csv`


 **Shapefiles**

 * Brooklyn Census Blocks: `../Data/census blocks only Brooklyn.csv`

 * Brooklyn Census Tracts: `../Data/census tracts Brooklyn.csv`

 * Brooklyn Zip Codes: `../Data/zip code Brooklyn.csv`

#### Output Files:

 * Amnesty International Cameras in Brookyln: `../Data/amnesty_international_cameras_brooklyn.csv`

 * Hikvision Cameras in Brooklyn: `../Data/hikvision_camera_brooklyn.csv`

In [2]:
#load camera data
path = os.getcwd()
parent = os.path.dirname(os.path.dirname(path))
amnesty = pd.read_csv('../Data/amnesty_camera_counts_per_intersections.csv')
hikvision = pd.read_csv('../Data/NYC_Surveillance_Map_Hikvision.csv', index_col = 0)

FileNotFoundError: [Errno 2] No such file or directory: '../Data/amnesty_camera_counts_per_intersections.csv'

In [ ]:
amnesty.head()

In [ ]:
hikvision.head()

In [ ]:
#load shapefiles
nyc_blks = pd.read_csv('../Data/2020_Census_Blocks_-_Tabular.csv')
nyc_tracts = pd.read_csv('../Data/2020_Census_Tracts_-_Tabular.csv')
nyc_blks.head()

In [ ]:
nyc_tracts.head()

In [ ]:
# Convert the 'the_geom' column from WKT format to Shapely geometry objects
nyc_blks['the_geom'] = nyc_blks['the_geom'].apply(loads)
nyc_tracts['the_geom'] = nyc_tracts['the_geom'].apply(loads)

# Convert the pandas DataFrame to a GeoDataFrame
blks_gdf = gpd.GeoDataFrame(nyc_blks, geometry='the_geom', crs = '4326')
tracts_gdf = gpd.GeoDataFrame(nyc_tracts, geometry='the_geom', crs = '4326')

In [ ]:
blks_gdf.head()

In [ ]:
print(len(blks_gdf), len(tracts_gdf))

In [ ]:
bk_blks = blks_gdf[blks_gdf['BoroName'] == 'Brooklyn'].reset_index(drop = True)
bk_tracts = tracts_gdf[tracts_gdf['BoroName'] == 'Brooklyn'].reset_index(drop = True)

In [ ]:
print(len(bk_blks),len(bk_tracts))

In [ ]:
#plot
bk_blks.plot()

In [ ]:
bk_tracts.plot(figsize = (10,10))

In [ ]:
#get the boundary geometry
from shapely.ops import unary_union

boundary = unary_union(bk_tracts.geometry)
bndy_gdf = boundary_gdf = gpd.GeoDataFrame(geometry=[boundary], crs='EPSG:4326')

In [ ]:
bndy_gdf.plot()

In [ ]:
#covert the geometry points in the camera datasets and filter via spatial join
from shapely.geometry import Point

# Create a new column with Point geometries based on the 'lat' and 'lon' columns
geometry = [Point(xy) for xy in zip(amnesty['Long'], amnesty['Lat'])]

# Create a GeoDataFrame with the 'geometry' column and CRS set to EPSG:4326
amnesty_gdf = gpd.GeoDataFrame(amnesty, geometry=geometry, crs='EPSG:4326')
amnesty_gdf.head()

In [ ]:
#spatial join
# Conduct a spatial join between 'amnesty_gdf' and the boundary polygon
amnesty_bk = gpd.sjoin(amnesty_gdf, bndy_gdf, how='inner', predicate='within')
print(len(amnesty_gdf),len(amnesty_bk))

In [ ]:
#repeat for the hikvision df
hikvision['wkt'] = hikvision['wkt'].apply(loads)

# Convert the pandas DataFrame to a GeoDataFrame
hikvision_gdf = gpd.GeoDataFrame(hikvision, geometry='wkt', crs = '4326')
hikvision_bk = gpd.sjoin(hikvision_gdf, bndy_gdf, how='inner', predicate='within')
print(len(hikvision_gdf),len(hikvision_bk))

In [ ]:
amnesty_bk.head()

In [ ]:
hikvision_bk = hikvision_bk.drop(columns = ['index_right']).reset_index(drop = True)
hikvision_bk

In [ ]:
amnesty_bk = amnesty_bk.drop(columns = ['index_right']).reset_index(drop = True)
amnesty_bk

In [ ]:
#amnesty_bk.to_csv('../Data/amnesty_international_cameras_brooklyn.csv')

In [ ]:
#hikvision_bk.to_csv('../Data/hikvision_camera_brooklyn.csv')

In [ ]:
amnesty_bk_agg = amnesty_bk.groupby(['Lat', 'Long']).agg(
        {"n_cameras_median": "sum", 
         "n_cameras_agreement": "sum"}
).reset_index()

In [ ]:
amnesty_bk_agg

In [ ]:
# Aggregate counts by geographic location
hikvision_bk_agg = hikvision_bk.groupby(['Latitude', 'Longitude']).sum()
hikvision_bk_agg